# Sliding Tile Programming Practice!

In this project, we will explore the [sliding tile puzzle](https://en.wikipedia.org/wiki/Sliding_puzzle). The sliding tile puzzle is a game similar to the rubicks cube, where the goal is to place the numbered squares in order, given a random initial state.

## Importing External Libraries

In this section, we will import all the external libraries needed to successfully complete the programming problem.

You do __NOT__ have to do anything here! In fact, please do __NOT__ modify anything in this section.

In [ ]:
#### SETUP CODE #####
### DO NOT MODIFY ###
!pip install --upgrade --force-reinstall --quiet git+https://github.com/chrisliu/lacc2023-module3-ml.git
from lacc.sliding_tile.sliding_tile import SlidingTile, generate_valid_sliding_tile, middle_state, goal_state, AStarSearch
print('Done with External Imports!')
#### SETUP CODE #####
### DO NOT MODIFY ###

## Getting Acquainted With the Puzzle

Now, we have imported everything, its time to test everything and see how it all works.

Firstly, let's see what a "sliding tile" looks like:

In [ ]:
puzzle = middle_state(3) # Generate a random puzzle of size 3x3.
print(puzzle) # Print this puzzle to the output

The function above generated a random sliding tile puzzle. As you can see, there are nine tiles with eight being labeled as "1-8" and one blank tile.

During this game, we can move the various tiles. Let's see what that looks like now.

In [ ]:
puzzle.print_actions('drul') # Print the sequence of actions down, right, up, left

As you can see in this example, we pushed the blank tile down, then left, then up, then right. During the sliding tile puzzle, you can move the blank tile in any direction, as long as its not removed from the board.

The goal of the sliding tile puzzle is to find a sequence of moves such that the sliding tile will look like the goal state. What does the goal state look like? Well, let's see!

In [ ]:
goal = goal_state(3) # Get a goal state for a 3x3 puzzle
print('goal:', goal, 'puzzle:', puzzle, sep='\n') # Print the puzzle and goal side by side

As you can see, it will usually take a few moves to get to the goal state. An additional goal by computer scientists, is to find the fewest number of moves possible to solve the puzzle.

How is this puzzle represented in the computer? Well, its usually represented by a 2-dimensional list with the blank tile representing a zero. Let's print this out:

In [ ]:
print('puzzle:', puzzle, 'puzzle (as array):', puzzle.to_array(), sep='\n')

Thus, you can edit the puzzle just like an array. This means you can modify the puzzle accordingly.

For instance, let's get the middle tile (or blank tile in this case):

In [ ]:
print('puzzle:', puzzle, 'middle tile:', puzzle[1][1], 'bottom corner tile:', puzzle[2][2], sep='\n')

Now that you can see how the puzzle works, lets start finding a way to solve it!

## Implementing Actions

Now, our first task is implementing the different actions that we can perform. As showcased above, we must be able to slide the blank tile up, down, left, and right. We can do this by modifying the underlying array associated with the sliding tile puzzle.

For instance, let's look at a working "slide_up" function.

In [ ]:
def slide_up(puzzle):
    new_puzzle = puzzle.copy() # copy puzzle so that we aren't editing the old puzzle

    # First, get location of the blank tile
    x_blank_tile, y_blank_tile = puzzle.blank_tile()

    # Second, let's check if its possible to move upward
    if x_blank_tile == 0:
        return False # Return False because moving upward is not allowed in this case

    # Now lets swap the empty tile with the tile above it
    tile_above = puzzle[x_blank_tile - 1][y_blank_tile]
    new_puzzle[x_blank_tile - 1][y_blank_tile] = 0
    new_puzzle[x_blank_tile][y_blank_tile] = tile_above

    # Let's return this new modified puzzle
    return new_puzzle

As you can see in the code, there were three steps to the slide up action:

1. Get the location of the blank tile
2. Check to see if its possible to slide upward
3. Swap the blank tile with its upward neighbor.

As always, let us test our code to see if it works.

In [ ]:
puzzle = middle_state(3)
print('base puzzle:', puzzle, 'slide_up:', slide_up(puzzle), 'slide_up twice:', slide_up(slide_up(puzzle)), sep='\n')


Your job is to now implement the slide_down, slide_left, and slide_right functions based upon the function provided.

In [ ]:

def slide_down(puzzle):
      new_puzzle = puzzle.copy() # copy puzzle so that we aren't editing the old puzzle

      ''' Start of Answer '''

      # First, get location of the blank tile
      x_blank_tile, y_blank_tile = puzzle.blank_tile()

      # Second, let's check if its possible to move down
      if x_blank_tile == puzzle.shape[1] - 1:
          return False

      # Third, lets swap the empty tile with the tile below it
      tile_below = puzzle[x_blank_tile + 1][y_blank_tile]
      new_puzzle[x_blank_tile + 1][y_blank_tile] = 0
      new_puzzle[x_blank_tile][y_blank_tile] = tile_below

      ''' End of Answer '''

      return new_puzzle

def slide_left(puzzle):
      new_puzzle = puzzle.copy() # copy puzzle so that we aren't editing the old puzzle

      ''' Start of Answer '''

      # First, get location of the blank tile
      x_blank_tile, y_blank_tile = puzzle.blank_tile()

      # Second, let's check if its possible to move down
      if y_blank_tile == 0:
          return False

      # Third, lets swap the empty tile with the tile below it
      tile_to_left = puzzle[x_blank_tile, y_blank_tile - 1]
      new_puzzle[x_blank_tile, y_blank_tile - 1] = 0
      new_puzzle[x_blank_tile, y_blank_tile] = tile_to_left

      ''' End of Answer '''

      return new_puzzle

def slide_right(puzzle : SlidingTile):
      new_puzzle = puzzle.copy() # copy puzzle so that we aren't editing the old puzzle

      ''' Start of Answer '''

      # First, get location of the blank tile
      x_blank_tile, y_blank_tile = puzzle.blank_tile()

      # Second, let's check if its possible to move down
      if y_blank_tile == puzzle.shape[0] - 1:
          return False

      # Third, lets swap the empty tile with the tile below it
      tile_to_right = puzzle[x_blank_tile, y_blank_tile + 1]
      new_puzzle[x_blank_tile, y_blank_tile + 1] = 0
      new_puzzle[x_blank_tile, y_blank_tile] = tile_to_right

      ''' End of Answer '''

      return new_puzzle

Now, let's test your implementations. You will need to visually inspect to see if this works.

In [ ]:
puzzle = middle_state(3)

print('base puzzle:', puzzle, sep='\n')
print('slide_down:', slide_down(puzzle), 'slide_down twice:', slide_down(slide_down(puzzle)), sep='\n')
print('slide_left:', slide_left(puzzle), 'slide_left twice:', slide_left(slide_left(puzzle)), sep='\n')
print('slide_right:', slide_right(puzzle), 'slide_right twice:', slide_right(slide_right(puzzle)), sep='\n')


## Implementing the goal_state

Now that we have our actions, we now need a function to check whether our sliding tile puzzle has reached the goal state.

In [ ]:
def check_solved(puzzle : SlidingTile) -> bool:
      ''' Checks to see if the puzzle is solved
          Input: the current sliding tile puzzle
          Output: True if the puzzle is solved, False otherwise
      '''

      ''' Start of Answer '''
      goal = goal_state(len(puzzle))

      for i in range(len(puzzle)):
          for j in range(len(puzzle[i])):
              if puzzle[i, j] != goal[i][j]:
                  return False # We did not reach goal
      return True
      ''' End of Answer '''

As with before, let's test to see if our goal state works! You will need to visually inspect this case:

In [ ]:
random_puzzle = generate_valid_sliding_tile(3)
goal = goal_state(3)

print('Random sliding tile puzzle:', random_puzzle, 'is_goal: ' + str(check_solved(random_puzzle)), sep='\n')
print('Goal sliding tile puzzle:', goal, 'is_goal: ' + str(check_solved(goal)), sep='\n')


## Depth First Search

If you remember from earlier in the day, the first search method that we looked at was depth first search. This search method will search seek to explore one path as far as possible before seeking to explore another path. While this won't get us an optimal solution, we will quickly get a solution that works.

So, let's get coding:

In [ ]:
def depth_first_search(puzzle : SlidingTile):
  ''' Performs a Breadth First Search of the Sliding Tile Puzzle
      Input: A starting SlidingTile Puzzle
      Output: A sequence of moves that will solve the puzzle
  '''

  # First, let's create a list that will contain all of the puzzles that we've already seen.
  seen_puzzles = []

  # Second, let's create an array of all the 'states' that we need to explore. This
  # will contain both the current puzzle state and actions to reach this state
  to_explore = [('', puzzle)] # We populate this array with our starting state

  while len(to_explore) > 0: # Keep looping while there's still spaces to explore
    # Get the current state to explore
    path, cur_puzzle = to_explore.pop() # This will get the most recent puzzle

    # Check to see if we've already seen this puzzle
    if cur_puzzle in seen_puzzles:
      continue

    # As we haven't seen this puzzle, let's add it to the list of seen states
    seen_puzzles.append(cur_puzzle)

    # Now, let's check to see if the current puzzle is a solution.
    # If it is, we should return the path to get to this current puzzle
    ''' Start of Solution'''
    if check_solved(cur_puzzle):
      return path
    ''' End of Solution'''

    # Now, we know that we don't have a solution. Thus, we need to add the next
    # states to the list of states to explore. We must do this for each possible
    # Action. Let's start with slide_up

    up = slide_up(cur_puzzle) # Get the puzzle where we slide up
    if up is not False: # Check to make sure this is a valid move
      to_explore.append((path + 'u', up)) # Now let's add this to list of next states


    # Your job is now add the slide_down, slide_left, and slide_right actions
    ''' Start of Solution '''
    down = slide_down(cur_puzzle) # Get the puzzle where we slide down
    if down is not False: # Check to make sure this is a valid move
      to_explore.append((path + 'd', down)) # Now let's add this to list of next states

    left = slide_left(cur_puzzle) # Get the puzzle where we slide left
    if left is not False: # Check to make sure this is a valid move
      to_explore.append((path + 'l', left)) # Now let's add this to list of next states

    right = slide_right(cur_puzzle) # Get the puzzle where we slide right
    if right is not False: # Check to make sure this is a valid move
      to_explore.append((path + 'r', right)) # Now let's add this to list of next states
    ''' End of Solution '''

    # Now that we finished all possible actions, we can go to next loop and are done

  raise ValueError('No Possible Solution for puzzle:\n', puzzle)

Now that we have a search function, lets test that it works.

In [ ]:
random_puzzle = SlidingTile(3, [[1, 2, 3], [4, 5, 0], [7, 8, 6]])
print('Puzzle:', random_puzzle, sep='\n')
print('Finding Solution......')
solution_dfs = depth_first_search(random_puzzle)
print('Solution Found!', solution_dfs)
print('Solution is at depth', len(solution_dfs))
print('printing solution....')
random_puzzle.print_actions(solution_dfs)

If you were able to find a solution (with a depth of ~29) for the above example, then you're solution works!

## Breadth First Search

Now, let's try implementing the second algorithm, Breadth First Search. This algorithm will seek to explore all possible actions before going to the next solution. Importantly, Breadth First Search will find an optimal solution, often at the expense of time compared to Depth First Search.

Now, let's program Breadth First Search. It will be very similar to Depth First Search.

In [ ]:
def breadth_first_search(puzzle : SlidingTile):
  ''' Performs a Breadth First Search of the Sliding Tile Puzzle
      Input: A starting SlidingTile Puzzle
      Output: A sequence of moves that will solve the puzzle
  '''

  # First, let's create a list that will contain all of the puzzles that we've already seen.
  seen_puzzles = []

  # Second, let's create an array of all the 'states' that we need to explore. This
  # will contain both the current puzzle state and actions to reach this state
  to_explore = [('', puzzle)] # We populate this array with our starting state

  while len(to_explore) > 0: # Keep looping while there's still spaces to explore
    # Get the current state to explore
    path, cur_puzzle = to_explore.pop(0) # This will get the most recent puzzle

    # Check to see if we've already seen this puzzle
    if cur_puzzle in seen_puzzles:
      continue

    # As we haven't seen this puzzle, let's add it to the list of seen states
    seen_puzzles.append(cur_puzzle)

    # Now, let's check to see if the current puzzle is a solution.
    # If it is, we should return the path to get to this current puzzle
    ''' Start of Solution'''
    if check_solved(cur_puzzle):
      return path
    ''' End of Solution'''

    # Now, we know that we don't have a solution. Thus, we need to add the next
    # states to the list of states to explore. We must do this for each possible
    # Action. Let's start with slide_up

    up = slide_up(cur_puzzle) # Get the puzzle where we slide up
    if up is not False: # Check to make sure this is a valid move
      to_explore.append((path + 'u', up)) # Now let's add this to list of next states


    # Your job is now add the slide_down, slide_left, and slide_right actions
    ''' Start of Solution '''
    down = slide_down(cur_puzzle) # Get the puzzle where we slide down
    if down is not False: # Check to make sure this is a valid move
      to_explore.append((path + 'd', down)) # Now let's add this to list of next states

    left = slide_left(cur_puzzle) # Get the puzzle where we slide left
    if left is not False: # Check to make sure this is a valid move
      to_explore.append((path + 'l', left)) # Now let's add this to list of next states

    right = slide_right(cur_puzzle) # Get the puzzle where we slide right
    if right is not False: # Check to make sure this is a valid move
      to_explore.append((path + 'r', right)) # Now let's add this to list of next states
    ''' End of Solution '''

    # Now that we finished all possible actions, we can go to next loop and are done

  raise ValueError('No Possible Solution for puzzle:\n', puzzle)

Now, let's test this code.

In [ ]:
random_puzzle = SlidingTile(3, [[2, 3, 0], [1, 4, 6], [7, 5, 8]])
print('Puzzle:', random_puzzle, sep='\n')
print('Finding Solution......')
solution_bfs = breadth_first_search(random_puzzle)
print('Solution Found!', solution_bfs)
print('Solution is at depth', len(solution_bfs))
print('printing solution....')
random_puzzle.print_actions(solution_bfs)

## Comparing Breadth-First to Depth-First


Now that we have both Breadth-First and Depth-First search, lets compare them against each other. What do you notice (time, solution, etc)

In [ ]:
random_puzzle = SlidingTile(3, [[1, 2, 3], [4, 5, 0], [7, 8, 6]])
print('Puzzle:', random_puzzle, sep='\n')
breadth_first_solution = breadth_first_search(random_puzzle)
print('Breadth First Solution', breadth_first_solution)
depth_first_solution = depth_first_search(random_puzzle)
print('Depth First Solution', depth_first_solution)



## Advanced Section: A* Search

If you have completed the other sections, this section is a little harder but can be considered a bonus. You aren't expected to complete this section.

### Motivation
As you may have noticed, even though we are generating the correct solution it sometimes takes a very long time and won't even finish at all.

Let's take the following example

In [ ]:
sliding_tile = SlidingTile(3, [[4, 2, 3], [7, 0, 5], [8, 6, 1]])
solved_sliding_tile = breadth_first_search(sliding_tile)
print('Sliding Tile Sample:\n', sliding_tile, '\nSolution:', solved_sliding_tile)

If you have a computer like ours, this example will take really long to finish. Why is that?

Well, since we are exhaustively searching (using breadth-first-search) we must do every combination of actions before adding an action. This means that solutions that have larger depths (or require many actions) will take a very long time to complete, or may not complete at all.

Thus, we have to be a little smarter about how we search. We can't search everything because it takes to long.

A method to limit how long the searching takes is by using a heuristic. If we can judge how many more actions it will take for us to reach the solution, then we can search these paths first. Something to note is that this heuristic must be optimistic, or underestimate the number of moves it will take, as then we can still be guranteed to have the shortest possible path.

### Creating A Heuristic

Lets experiment with creating a first heuristic for this problem. For this heuristic, we are just going to count the number of spots that are wrong. This underestimates the number of moves, as we must move these tiles in order to get a solution.

For example:

>  [2, 3]

>  [0, 1]

will return 2. As 2,3 are in the right place and 0,1 are in the wrong place.

In [ ]:
def out_of_place(puzzle):
    ''' Calculates the number of tiles out of place for the given puzzle
        Input: Sliding Tile Puzzle
        Output: A number representing how many tiles are out of place
    '''
    num_out_of_place = 0
    ''' Start of Answer '''

    goal = goal_state(len(puzzle))

    for i in range(len(puzzle)):
        for j in range(len(puzzle[i])):
            if puzzle[i, j] != goal[i, j]:
                num_out_of_place += 1

    ''' End of Answer '''
    return num_out_of_place

Now, let's test our out-of-place heuristic

In [ ]:
goal = goal_state(3)

print('Goal State:', goal, sep='\n')
print('Goal State got', out_of_place(goal), 'should be', 0)

other_puzzle = SlidingTile(3, [[3, 2, 7], [4, 8, 1], [0, 6, 5]])
print('Random State:', other_puzzle, sep='\n')
print('Random State got', out_of_place(other_puzzle), 'should be', 7)

### A* Search

Now that we have a heuristic, lets use it in our search algorithm. This algorithm is called A*-search, as it considers how far we have been and how far we must go to arrive at the solution.

It follows the formula, F(N) = G(N) + H(N)

where, F is the estimate of the final distance, G is the current distance that we calculated, and H is the estimated further distance that we must go.

A* will search states with the lowest final distance first, before those with the longer distance. To do this, we are using a data structure called heap.

As this is a little more complicated, we have fully implemented it below

Now let's solve the puzzle we had from earlier

In [ ]:
sliding_tile = SlidingTile(3,[[4, 2, 3], [7, 0, 5], [8, 6, 1]])
solved_sliding_tile, expanded = AStarSearch(sliding_tile, out_of_place)
print('Sliding Tile Sample:\n', sliding_tile, '\nSolution:', solved_sliding_tile, ' Expanded:', expanded)

### Custom Heuristic

However, our heuristic only goes so far. You may still notice that we can't solve a random three-by-three sliding tile puzzle yet. For the rest of this project, your job is to find a heuristic function that can beat our simple out_of_place count.

What other estimates can you find that could estimate how much further better than we can?

In [ ]:
def your_heuristic(puzzle):
    ''' Start of Answer '''
    '''
    Answers may vary. Possible strategies include:
    1. Count the number of tiles out of place
    2. L1 Distance
    3. L1 Distance + Linear Conflict
    4. Pattern Table

    I would only expect them to come up with L1 Distance. Make sure that solution is always an underestimate.

    '''
    pass
    ''' End of Answer '''

sliding_tile = generate_valid_sliding_tile(4)
solved_sliding_tile, expanded = AStarSearch(sliding_tile, your_heuristic)
print('Sliding Tile Sample:\n', sliding_tile, '\nSolution:', solved_sliding_tile, ' Expanded:', expanded)